In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from os.path import exists
from os import remove
import sklearn.linear_model as lm
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import sklearn.model_selection as ms

import sys
sys.path.insert(0, "../util/")
import util as util

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Add Dummies and Train-Test-Split

In [2]:
housingTrainWithDummiesExists = exists('../data/housingTrainWithDummies.csv')
housingTestWithDummiesExists = exists('../data/housingTestWithDummies.csv')
housingTrainWithoutDummiesExists = exists('../data/housingTrainWithoutDummies.csv')
housingTestWithoutDummiesExists = exists('../data/housingTestWithoutDummies.csv')
relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

if (housingTrainWithDummiesExists and housingTestWithDummiesExists and
   housingTrainWithoutDummiesExists and housingTestWithoutDummiesExists and
   relatedDummiesDictionaryExists):
    housingTrainWithDummies = pd.read_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies = pd.read_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies = pd.read_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies = pd.read_csv('../data/housingTestWithoutDummies.csv')
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    
else:
    if (housingTrainWithDummiesExists):
        remove('../data/housingTrainWithDummies.csv')
    if (housingTestWithDummiesExists):
        remove('../data/housingTestWithDummies.csv')
    if (housingTrainWithoutDummiesExists):
        remove('../data/housingTrainWithoutDummies.csv')
    if (housingTestWithoutDummiesExists):
        remove('../data/housingTestWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    amesHousing = pd.read_csv('../data/Ames_Housing_Price_Data.csv')   
    
    amesHousing['MSSubClass'] = amesHousing['MSSubClass'].apply(str)
    
    amesHousingCategoricalData = amesHousing.select_dtypes(include=['O'])
    for col in amesHousingCategoricalData.columns:
        amesHousing[col]=amesHousing[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in amesHousingCategoricalData.columns:
        dummyData = pd.get_dummies(amesHousing[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
        amesHousing = pd.concat([amesHousing,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
    
    salesPrice = amesHousing['SalePrice']
    X = amesHousing.drop(['SalePrice'], axis=1)
    
    housingTrainWithDummies, housingTestWithDummies, salesPriceTrain, salesPriceTest = train_test_split(X, salesPrice, test_size=0.2)
    
    housingTrainWithDummies['SalePrice'] = salesPriceTrain
    housingTestWithDummies['SalePrice'] = salesPriceTest
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    housingTrainWithoutDummies = housingTrainWithDummies.drop(dummyColumns, axis=1)
    housingTestWithoutDummies = housingTestWithDummies.drop(dummyColumns, axis=1)
    
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies.to_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies.to_csv('../data/housingTestWithoutDummies.csv')

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
if (("ISUDistance" not in housingTrainWithoutDummies.columns) or 
    ("ISUDistance" not in housingTrainWithDummies.columns)):
    housingTrainWithoutDummies = util.returnDFWithISUDistance(housingTrainWithoutDummies,True)
    housingTrainWithDummies["ISUDistance"] = housingTrainWithoutDummies["ISUDistance"]
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')

# Training

## Linear Regression

In [4]:
housingTrainNumerical = housingTrainWithDummies.select_dtypes(include=['uint8','int64','float64'])

In [5]:
if (exists('../data/trainNanReplacementValuesDictionary.json')):
    with open('../data/trainNanReplacementValuesDictionary.json') as d:
        trainNanReplacementValuesDictionary = json.load(d)
else:
    trainNanReplacementValuesDictionary = {}
    for col in housingTrainNumerical:
        if col in ["YearBuilt","YearRemodAdd","GarageYrBlt","YrSold","ISUDistance"]:
            trainNanReplacementValuesDictionary[str(col)] = housingTrainNumerical[col].mean()
        elif col in ["OverallQual","OverallCond","MoSold"]:
            trainNanReplacementValuesDictionary[str(col)] = round(np.mean(housingTrainNumerical[col].mode().values))
        else:
            trainNanReplacementValuesDictionary[str(col)] = 0
    with open('../data/trainNanReplacementValuesDictionary.json', 'w') as fp:
        json.dump(trainNanReplacementValuesDictionary, fp)
    

In [6]:
housingTrainNumerical = util.replaceNansWithTrainingDataValues(housingTrainNumerical)

../util/util.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(trainNanReplacementValuesDictionary[str(col)])


In [7]:
print(f'{housingTrainNumerical.shape[0]} rows before removing outliers')
housingTrainNumerical = util.removeOutliers(housingTrainNumerical)
print(f'{housingTrainNumerical.shape[0]} rows after removing outliers')

2064 rows before removing outliers
2024 rows after removing outliers


In [8]:
salesPrice = housingTrainNumerical["SalePrice"]
X = housingTrainNumerical.drop(['SalePrice'], axis=1)

In [9]:
initialAlpha = [1]+list(np.linspace(50,1000,20))

In [10]:
grid_para_elsatic_net_initial = {'alpha': initialAlpha, 'l1_ratio': np.linspace(0,1,11)}

In [11]:
elasticNet = ElasticNet()
if False:
    grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_initial, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
    grid_search_elastic_net.fit(X,salesPrice)
    grid_search_elastic_net.best_params_

In [12]:
finalAlpha = [1] + list(np.linspace(5,50,10))

In [13]:
grid_para_elsatic_net_final = {'alpha': finalAlpha, 'l1_ratio': [0.0]}

In [14]:
grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_final, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
grid_search_elastic_net.fit(X,salesPrice)
grid_search_elastic_net.best_params_

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 737284073046.4095, tolerance: 1124719653.9471476
  model = cd_fast.enet_coordinate_descent(


{'alpha': 1, 'l1_ratio': 0.0}

## Random Forest

# Testing

## Linear Regression

## Random Forest